In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import linear_model
import random
from sklearn import feature_selection
import statsmodels.formula.api as smf
from sklearn import metrics

Let's assume here is your actual output
y = [1,2,3,4,5]

And your prediction is

y_predict = [2,2,3,4,5]

Your MSE is ((1-2)^2 + (2-2)^2 + ... + (5-5)^2)/5 = 1/5 = .2

In [ ]:
#Let's find MSE using sklearn 

y = [1,2,3,4,5]
y_predict = [2,2,3,4,5]
metrics.mean_squared_error(y,y_predict)

In [ ]:
#Let's generate 300 random numbers between -10 and 10
x = np.zeros(300)
for i in range(300):
   x[i] = random.uniform(-10, 10)

#Let's generate some error term with mean 0 and s.d. = 20
error = np.random.normal(0,20,300)
#Now let's generate y with a polynomial degree 2 relationship with x

y = 3 + 1.5 * x + 4 * (x ** 2) + error
df = pd.DataFrame({'X': x, 'y': y})

df.plot(kind = 'scatter', x = 'X', y = 'y')

In [ ]:
#Now let's add few non-linear terms to our data frame
#The 'correct' model will select X and X^2 values
df['X_2'] = df.X ** 2
df['X_3'] = df.X ** 3
df['X_4'] = df.X ** 4
df['X_5'] = df.X ** 5

df.head(2)

In [ ]:
X1 = df[['X']]
X2 = df[['X','X_2']]
X3 = df[['X','X_2','X_3']]
X4 = df[['X','X_2','X_3','X_4']]
X5 = df[['X','X_2','X_3','X_4','X_5']]
y = df['y']

In [ ]:
lm = linear_model.LinearRegression()
MSE = np.zeros(5)
j = 0
for i in [X1,X2,X3,X4,X5]:
    lm.fit(i,y)
    MSE[j] = (metrics.mean_squared_error(lm.predict(i),y))
    pvals = feature_selection.f_regression(i,y)[1]
    print(pvals)
    j += 1



#### in comparison to R, sklearn is using a different method to estimate p-values. P-values in sklearn are not reliable. If you want to use p-values, use statsmodels.formula.api

In [ ]:
for i in [X1,X2,X3,X4,X5]:
    lm1 = smf.ols(formula='y ~ i', data=df).fit()
    print(lm1.summary())


#### Based on P-values, it is clear that our models are only valid up to 2 degrees of polynomial. After that coefficients are not statistically significant. 

# Validation

As much as sklearn performs weak for p-values, it is a great tool for splitting data into Training and Test set. 

In [ ]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.33)
#We train based on Training Data BUT will Test on Test Sample
Model_train = lm.fit(X_train,y_train)
y_Hat_train = lm.predict(X_train)
y_Hat_test  = lm.predict(X_test)
MSE_Train = metrics.mean_squared_error(y_Hat_train,y_train)
MSE_Test  = metrics.mean_squared_error(y_Hat_test,y_test)
print("MSE_Train =",MSE_Train)
print("MSE_Test =", MSE_Test)


Now, it's time to use Validation techniques to test which model is more significant. Remember we need to train based on training sets but test the performance on test set.

In [ ]:
MSE_test = np.zeros(5)
MSE_train = np.zeros(5)
j = 0
for i in [X1,X2,X3,X4,X5]:
    X_train, X_test, y_train, y_test = train_test_split(i, y, test_size=0.33)
    lm.fit(X_train,y_train)
    MSE_test[j] = (metrics.mean_squared_error(lm.predict(X_test),y_test))
    MSE_train[j] = (metrics.mean_squared_error(lm.predict(X_train),y_train))
    j += 1

index = np.array(range(5)) + 1
MSE_Test_df = pd.DataFrame({'MSE_test':MSE_test,'index':index,'MSE_train':MSE_train})
print(MSE_Test_df)
MSE_Test_df.plot(x = 'index',y= ['MSE_test','MSE_train'])


Based on test-set MSE, we decide to choose Model 2 - polynomial degree 2. Remember ** THE SIMPLER THE BETTER **.

# Cross-Validation

sklearn is great for Cross-Validation. 10 fold and 5 fold cross-validation are the most famous cross-validation techniques. 

In [ ]:
from sklearn import cross_validation
kf = cross_validation.KFold(len(df), n_folds = 5, shuffle = True) 
#shuffle=True randomly creates groups rather than keeping them in order
#creates a list of tuples

#for train_index, test_index in kf:
#    print("TRAIN:", train_index, "TEST:", test_index) #yields two arrays of indices?

scores = []
for train_index, test_index in kf:
    lm = linear_model.LinearRegression().fit(X2.iloc[train_index], y.iloc[train_index])
    scores.append(metrics.mean_squared_error(y.iloc[test_index], lm.predict(X2.iloc[test_index])))

print('Raw scores:',scores)
print('Mean:',np.mean(scores))
 
  

#### Now, let's use cross validation to decide which degree of polynomials is suitable for our simulated model. 

In [ ]:
kf = cross_validation.KFold(len(df), n_folds = 5, shuffle = True) 
MSE_CV = []

for i in [X1,X2,X3,X4,X5]:
    scores = []
    for train_index, test_index in kf:
        lm = linear_model.LinearRegression().fit(i.iloc[train_index], y.iloc[train_index])
        scores.append(metrics.mean_squared_error(y.iloc[test_index], lm.predict(i.iloc[test_index])))
    MSE_CV.append(np.mean(scores))
        
        
print(MSE_CV)
index = np.array(range(5)) + 1
MSE_CV_df = pd.DataFrame({'MSE_CV':MSE_CV,'index':index})
MSE_CV_df.plot(x = 'index',y= 'MSE_CV')

Again, 5-fold cross-validation suggests that polynomial degree 2 is our best model. 

# Lasso and Ridge Regression

We are going to use Credit data. We will first add dummy variables and drop original qualitative values. 

In [ ]:
url = "https://raw.githubusercontent.com/ga-students/SF-DAT-20/master/Data/Credit.csv"
CreditData = pd.read_csv(url)
RaceDummy = pd.get_dummies(CreditData.Ethnicity, prefix = 'Race')
del RaceDummy['Race_African American']
GenderDummy = pd.get_dummies(CreditData.Gender, prefix = 'Gender')
del GenderDummy['Gender_ Male']  
MarriedDummy = pd.get_dummies(CreditData.Married, prefix = 'Married')
del MarriedDummy['Married_No']
StudentDummy = pd.get_dummies(CreditData.Student, prefix = 'Student')
del StudentDummy['Student_No']
CreditData = pd.concat([CreditData, RaceDummy,GenderDummy,MarriedDummy,StudentDummy], axis=1)



del CreditData['Unnamed: 0']
del CreditData['Gender']
del CreditData['Student']
del CreditData['Married']
del CreditData['Ethnicity']
CreditData.head(2)

Now we are going to divide our dataset - to output y and all inputs X.

In [ ]:
listOfAllVariables = CreditData.columns.values
print(listOfAllVariables)
X = CreditData[listOfAllVariables]
del X['Balance']
y = CreditData['Balance']


In [ ]:
#this was a lasso regression with default values
lm_lasso = linear_model.Lasso().fit(X,y)
lm_lasso.coef_

We need to use CV to decide on the optimal level of Alpha - the parameter of Lasso Regressions. 

In [ ]:
kf = cross_validation.KFold(len(CreditData), n_folds = 5, shuffle = True)
MSE_Lasso_CV = []
alphas = np.logspace(-10, 10, 21) #Getting 21 values between 1*10^-10 to 1*10^10
alphas_index = np.linspace(-10,10,21) #Define index as power of ten
for a in alphas:
    print 'Alpha:', a
    scores = []
    for train_index, test_index in kf:
        lm = linear_model.Lasso(alpha=a).fit(X.iloc[train_index], y.iloc[train_index]) #Creates model with coefficients
        scores.append(metrics.mean_squared_error(y.iloc[test_index], lm.predict(X.iloc[test_index])))
    MSE_Lasso_CV.append(np.mean(scores))



index = alphas
MSE_Lasso_CV_df = pd.DataFrame({'MSE_Lasso_CV': MSE_Lasso_CV ,'Log_Alphas': alphas_index })
MSE_Lasso_CV_df.plot(x = 'Log_Alphas',y = 'MSE_Lasso_CV')

In [ ]:
#MSE is flat up to log_alpha = 1. Log_alpha results in alpha = 10 (due to its logarithmic scale)
lm = linear_model.Lasso(alpha=10)
lm.fit(X, y)
print zip(lm.coef_,X.columns)


#### Based on Lasso regression outputs, we decide to eliminate Gender, Marital Status, Education, and Race from our model. The coefficients that made it to our final models were Income, Limit, Rating (we shall only choose 1 of these three variables due to colinearity), Number of Cards, Age, and Studentship. Based on our previous In-Class-Practice, we were expecting to have Income and Studentship in our model. Now, since we are only left with 4 variables, we can run statistical tests and choose the most significant model. 


#### Ridge Regression

In [ ]:
kf = cross_validation.KFold(len(CreditData), n_folds = 5, shuffle = True)
MSE_Ridge_CV = []
alphas = np.logspace(-10, 10, 21)
alphas_index = np.linspace(-10,10,21)
for a in alphas:
    #print 'Alpha:', a
    scores = []
    for train_index, test_index in kf:
        lm = linear_model.Ridge(alpha=a).fit(X.iloc[train_index], y.iloc[train_index])
        scores.append(metrics.mean_squared_error(y.iloc[test_index], lm.predict(X.iloc[test_index])))
    MSE_Ridge_CV.append(np.mean(scores))

            #print lm.coef_
        #MSE_Lasso_CV.append(metrics.mean_squared_error(y, lm.predict(X)))

index = alphas
MSE_Ridge_CV_df = pd.DataFrame({'MSE_Ridge_CV': MSE_Ridge_CV ,'log_alpha': alphas_index })
MSE_Ridge_CV_df.plot(x = 'log_alpha',y = 'MSE_Ridge_CV')

In [ ]:
#MSE is flat up to log_Alpha = 1. log_alpha = 1 results in alpha = 10 (due to its logarithmic scale)
lm = linear_model.Ridge(alpha=10)
lm.fit(X, y)
print zip(lm.coef_,X.columns)

#### Based on Ridge regression model, we are not comfortable to eliminate any variable. Generally, Lasso is much better than Ridge Regression.  

# Scaling / Standardizing Data

In [ ]:
from sklearn import preprocessing


In [ ]:
CreditDataNew = preprocessing.scale(CreditData) #CreditDataNew is now a numpy array
CreditDataNew = pd.DataFrame(CreditDataNew)   #We changed CreditDataNew to a dataframe
CreditDataNew.columns = CreditData.columns.values  #We renamed columns of CreditDataNew

In [ ]:
CreditDataNew.head()

In [ ]:
X = CreditDataNew[listOfAllVariables]
del X['Balance']
y = CreditDataNew['Balance']

In [ ]:
kf = cross_validation.KFold(len(CreditDataNew), n_folds = 5, shuffle = True)
MSE_Lasso_CV = []
alphas = np.logspace(-10, 10, 21)
alphas_index = np.linspace(-10,10,21)
scores = []
for a in alphas:
    print 'Alpha:', a
    scores = []
    for train_index, test_index in kf:
        lm = linear_model.Lasso(alpha=a).fit(X.iloc[train_index], y.iloc[train_index])
        scores.append(metrics.mean_squared_error(y.iloc[test_index], lm.predict(X.iloc[test_index])))
    MSE_Lasso_CV.append(np.mean(scores))



index = alphas
MSE_Lasso_CV_df = pd.DataFrame({'MSE_Lasso_CV': MSE_Lasso_CV ,'Log_Alphas': alphas_index })
MSE_Lasso_CV_df.plot(x = 'Log_Alphas',y = 'MSE_Lasso_CV')

In [ ]:
lm = linear_model.Lasso(alpha=10**(-2))
lm.fit(X, y)
print zip(lm.coef_,X.columns)

#### When you standardize data, then relative magnitude of coefficients are meaningful. Small Coefficients suggest insignificancy of inclusion of that variable in your model. In this case, Income, Limit, Rating and Student_status are considerably more significant than other variables. Among (Income, Limit and Rating) we only choose one due to high colinearity. Therefore, in are final model it is logical to keep (income, Limit, or Rating) + Student stauts. 